In [25]:
import csv
from datetime import datetime

In [37]:
def main():
    transactions = []
    while True:
        print("\nSmart Personal Finance Analyzer")
        print("1. Load Transactions")
        print("2. Add Transaction")
        print("3. View Transactions")
        print("4. Update Transaction")
        print("5. Delete Transaction")
        print("6. Analyze Finances")
        print("7. Save Transactions")
        print("8. Generate Report")
        print("9. Exit")
        choice = input("Select an option: ")
        if choice == '1':
            transactions = load_transactions()
            if transactions:
                print(f"Loaded {len(transactions)} transactions.")
            else:
                print("No transactions loaded or file not found.")
        elif choice == '2':
            try:
                print("Adding a new transaction...")
                add_transaction(transactions)
            except ValueError as e:
                print(f"Error adding transaction: {e}")
            except Exception as e:
                print(f"An unexpected error occurred: {e}")
        elif choice == '3':
            view_transactions(transactions)         
        elif choice == '4':
            transactions= update_transactions(transactions)
        elif choice == '5':
            transactions = delete_transactions(transactions)
        elif choice == '6':
            analyze_finances(transactions)
        elif choice == '7':
            save_transactions(transactions, 'financial_transactions.csv')
        elif choice == '8':
            generate_report(transactions)
        elif choice == '9':
            print("Exiting the program.")
            break
        else:
            print("Invalid choice. Please try again.")
